# ASHRAE Final Leaderboard
**Warning this is not the offical finals standings but based off of data provided by kaggle**

Medals are estimated using this kernel by @sheriytm:
https://www.kaggle.com/sheriytm/ashrae-public-lb-before-rescoring

The second shakeup visualization uses a design by @pednoi - please upvote his kernel here: https://www.kaggle.com/pednoi/visualize-the-shakeups-of-10-recent-competitions

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pylab as plt

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

pd.set_option('max_rows', 5000)

lb = pd.read_csv('/kaggle/input/ashrae-interim-scores/final_leaderboard_order.csv')

In [ ]:
lb['public_rank'] = lb['PublicScore'].rank().astype(int)
lb['private_rank'] = lb['PrivateScore'].rank().astype(int)
lb['shake'] = lb['public_rank'] - lb['private_rank']
lb['TeamName'] = lb['TeamName'].str[:20]
lb['medal'] = ''
lb.loc[lb['private_rank'] <= 367, 'medal'] = '🥉'
lb.loc[lb['private_rank'] <= 183, 'medal'] = '🥈'
lb.loc[lb['private_rank'] <= 17, 'medal'] = '🥇'
lb = lb[['private_rank','medal','shake',
         'TeamName','PublicScore','PrivateScore',
         'public_rank','SubmissionTimestamp']]

In [ ]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    try:
        color = 'red' if val < 0 else 'black'
    except:
        color = 'black'
    return 'color: %s' % color

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 8))
lb.query('PublicScore < 6 and PrivateScore < 6') \
    .plot(x='PublicScore',
        y='PrivateScore',
        kind='scatter',
        title='ASHRAE Shake < 6',
         ax=axes[0])
lb.query('PublicScore < 1.6 and PrivateScore < 1.6') \
    .plot(x='PublicScore',
        y='PrivateScore',
        kind='scatter',
        title='ASHRAE Shake < 1.6',
         ax=axes[1])
plt.show()

In [ ]:
def plot_candle(title, diff):
    closes = np.array(range(len(diff)))+1
    opens = closes + np.array(diff)
    highs = np.where(np.array(diff)<0, closes, opens)
    lows =  np.where(np.array(diff)>=0, closes, opens)
    
    hovertext = ['private lb: '+str(c)+'<br>public lb: '+str(o) for o, c in zip(opens, closes)]

    trace = go.Ohlc(x=list(range(1, len(diff)+1)), open=opens, high=highs, low=lows, close=closes,
                    increasing=dict(line=dict(color='#FF6699')), decreasing=dict(line=dict(color='#66DD99')),
                    text=hovertext, hoverinfo='text')
    
    layout = go.Layout(
        title = "<b>%s</b>" % title,
        xaxis = dict(
            title='final ranks',
            rangeslider = dict(visible=False)
        ), 
        yaxis=dict(
            title='shakeups',
            autorange='reversed'
        )
    )
    
    fig = go.Figure(data=[trace], layout=layout)    
    iplot(fig, filename='shakeup_candlestick')

plot_candle('ASHRAE Shakeup', lb['shake'])

In [ ]:
lb.style.\
    applymap(color_negative_red) \
    .hide_index()